In [ ]:
!pip install -q pytube pydub mutagen eyeD3

In [ ]:
import os
from pytube import YouTube
from pytube.exceptions import PytubeError
from pydub import AudioSegment
from mutagen.easyid3 import EasyID3
from mutagen.mp3 import MP3
import ipywidgets as widgets
from IPython.display import display

# Set Google Drive upload folder path
# Make sure you have mounted your Google Drive in Colab
# from google.colab import drive
# drive.mount('/content/drive')

drive_upload_folder = 'YouTube_MP3s'
drive_upload_folder_path = f'/content/drive/My Drive/{drive_upload_folder}'
os.makedirs(drive_upload_folder_path, exist_ok=True)
print(f"Google Drive upload folder set to: {drive_upload_folder_path}")

In [ ]:
# Paste multiple YouTube URLs here (one per line)
# You can also consider uploading a text file with URLs for a larger list.
youtube_urls_raw = """
https://youtu.be/ztq7-kkygZk?feature=shared

https://www.youtube.com/watch?v=dQw4w9WgXcQ

# Add more URLs here
"""

youtube_urls = [url.strip() for url in youtube_urls_raw.strip().splitlines() if url.strip()]

if not youtube_urls:
    print("No YouTube URLs found. Please paste URLs in the 'youtube_urls_raw' cell.")
else:
    print(f"Found {len(youtube_urls)} YouTube links:")
    for i, url in enumerate(youtube_urls, 1):
        print(f"{i}. {url}")

In [ ]:
# Adjustable clip length slider (in seconds)
# Set the desired length of the audio clip.
clip_length_slider = widgets.IntSlider(
    value=60, # Default clip length is 60 seconds
    min=10,
    max=600, # Increased max length to 10 minutes
    step=5,
    description='Clip Length (sec):',
    style={'description_width': 'initial'},
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(clip_length_slider)

In [ ]:
# Batch download, trim, tag, and save MP3s

failed_urls = []

for idx, url in enumerate(youtube_urls, 1):
    print(f"\n--- [{idx}/{len(youtube_urls)}] Processing: {url} ---")
    try:
        # Download audio
        print("  Downloading audio...")
        yt = YouTube(url)
        stream = yt.streams.filter(only_audio=True).first()
        if not stream:
             raise ValueError("No audio stream found for this video.")
        temp_filename = f"temp_audio_{idx}.webm"
        out_file = stream.download(output_path='.', filename=temp_filename)
        print("  Download complete.")

        # Trim and convert to MP3
        print("  Trimming and converting to MP3...")
        audio = AudioSegment.from_file(out_file)

        # Apply a short fade-out at the end (e.g., 1 second)
        fade_length = 1000 # milliseconds
        trimmed = audio[:clip_length_slider.value * 1000].fade(to_gain=-120.0, duration=fade_length)

        # Sanitize filename
        base_filename = yt.title
        # Remove characters that are not allowed in filenames
        safe_filename = ''.join(c for c in base_filename if c.isalnum() or c in (' ', '_', '-')).strip()
        # Replace spaces with underscores and limit length
        final_filename = f"{safe_filename[:100].replace(' ', '_')}.mp3"
        final_path = os.path.join(drive_upload_folder_path, final_filename)

        trimmed.export(final_path, format="mp3")
        print("  Trimming and conversion complete.")

        # Clean up temporary file
        os.remove(out_file)
        print("  Temporary file removed.")

        # Tag MP3
        print("  Tagging MP3...")
        try:
            audiofile = MP3(final_path, ID3=EasyID3)
            audiofile['title'] = yt.title
            audiofile['artist'] = yt.author
            audiofile['album'] = 'YouTube Batch Download'
            # Add more tags if desired, e.g.,
            # audiofile['genre'] = 'Unknown'
            # audiofile['date'] = str(yt.publish_date.year) if yt.publish_date else ''
            audiofile.save()
            print("  MP3 Tagging complete.")
        except Exception as tag_e:
            print(f"  ⚠️ Warning: Could not tag MP3 file {final_filename}: {tag_e}")
            # Continue processing even if tagging fails


        print(f"✅ Successfully processed and saved: {final_filename}\n  → {final_path}")

    except PytubeError as pe:
        print(f"❌ Error processing {url}: YouTube Error - {pe}")
        failed_urls.append(url)
    except FileNotFoundError:
        print(f"❌ Error processing {url}: File not found during processing.")
        failed_urls.append(url)
    except Exception as e:
        print(f"❌ An unexpected error occurred processing {url}: {e}")
        failed_urls.append(url)

print("\n--- Batch Processing Complete ---")
if failed_urls:
    print(f"⚠️ Failed to process {len(failed_urls)} URL(s):")
    for failed_url in failed_urls:
        print(f"- {failed_url}")
else:
    print("✅ All URLs processed successfully.")